## Simple Neural Nets with tensorflow

In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm

%load_ext autoreload
%autoreload
%config Completer.use_jedi=False

In [2]:
tf.executing_eagerly()

True

## Read in data

Using MNIST

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

Convert numpy tensors to tensorflow tensors and create batches

In [43]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Train with non customizable loop

Non customizable training loop

In [99]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation="sigmoid")
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [100]:
%%time
model.fit(train_dataset, epochs=50)

Train for 469 steps
Epoch 1/50
469/469 [==============================] - 1s 2ms/step - loss: 1.8584 - sparse_categorical_accuracy: 0.5448
Epoch 2/50
469/469 [==============================] - 1s 1ms/step - loss: 1.7462 - sparse_categorical_accuracy: 0.6691
Epoch 3/50
469/469 [==============================] - 1s 1ms/step - loss: 1.7258 - sparse_categorical_accuracy: 0.6869
Epoch 4/50
469/469 [==============================] - 1s 1ms/step - loss: 1.7129 - sparse_categorical_accuracy: 0.7127
Epoch 5/50
469/469 [==============================] - 1s 1ms/step - loss: 1.7045 - sparse_categorical_accuracy: 0.7214
Epoch 6/50
469/469 [==============================] - 1s 1ms/step - loss: 1.6523 - sparse_categorical_accuracy: 0.7567
Epoch 7/50
469/469 [==============================] - 1s 1ms/step - loss: 1.5627 - sparse_categorical_accuracy: 0.8412
Epoch 8/50
469/469 [==============================] - 1s 1ms/step - loss: 1.5493 - sparse_categorical_accuracy: 0.8594
Epoch 9/50
469/469 [========

In [101]:
model.evaluate(test_dataset)

79/79 [==============================] - 0s 2ms/step - loss: 1.5002 - sparse_categorical_accuracy: 0.9577


[1.5001950399785102, 0.9577]

Customizable training loop

In [109]:
tf.random.set_seed(42)

In [110]:
# Define simple neural net
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation="sigmoid")
])

# Define optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

# Define loss function
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_epoch_loss_avg = tf.keras.metrics.Mean()
test_epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [111]:
@tf.function
def train_loop(x, y):

    # Calculate gradients
    with tf.GradientTape() as t:
        # training=training is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(x, training=True)
        loss = loss_func(y, predictions)

    grads = t.gradient(loss, model.trainable_variables)

    # Optimize the model
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # Track progress
    epoch_loss_avg(loss)

    # Compare predicted label to actual
    epoch_accuracy.update_state(y, predictions)
    
#     return loss, predictions

In [112]:
%%time
train_loss_results = []
train_accuracy_results = []
epochs = 50
n_batches = len(list(train_dataset))

for epoch in tqdm(np.arange(epochs)):
    
    for x, y in tqdm(train_dataset, total=n_batches, leave=False):
        train_loop(x, y)

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    
    # Test
    for (x_valid, y_valid) in test_dataset:
        preds_test = model(x_valid, training=False)
        test_loss = loss_func(y_valid, preds_test)
        test_epoch_loss_avg(test_loss)
        test_epoch_accuracy.update_state(y_valid, preds_test)
    
    print(f"Epoch {epoch:03d}: train_loss: {epoch_loss_avg.result():.3f}, "
          f"test_loss: {test_epoch_loss_avg.result():.3f} "
          f"Accuracy: {epoch_accuracy.result():.3f}%"
          f"Test accuracy={test_epoch_accuracy.result():.3f}"
    )
    
    # Clear the current state of the metrics
    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()
    test_epoch_loss_avg.reset_states()
    test_epoch_accuracy.reset_states()
    # valid_loss.reset_states(), valid_acc.reset_states()
    
    

Epoch 000: train_loss: 1.786, test_loss: 1.637 Accuracy: 0.611%Test accuracy=0.787


Epoch 001: train_loss: 1.621, test_loss: 1.588 Accuracy: 0.790%Test accuracy=0.824


Epoch 002: train_loss: 1.585, test_loss: 1.569 Accuracy: 0.833%Test accuracy=0.845


Epoch 003: train_loss: 1.571, test_loss: 1.557 Accuracy: 0.852%Test accuracy=0.887


Epoch 004: train_loss: 1.559, test_loss: 1.550 Accuracy: 0.876%Test accuracy=0.883


...

Epoch 045: train_loss: 1.492, test_loss: 1.504 Accuracy: 0.956%Test accuracy=0.949


Epoch 046: train_loss: 1.491, test_loss: 1.502 Accuracy: 0.958%Test accuracy=0.950


Epoch 047: train_loss: 1.491, test_loss: 1.504 Accuracy: 0.958%Test accuracy=0.951


Epoch 048: train_loss: 1.490, test_loss: 1.501 Accuracy: 0.959%Test accuracy=0.951


Epoch 049: train_loss: 1.490, test_loss: 1.502 Accuracy: 0.958%Test accuracy=0.953

CPU times: user 1min 16s, sys: 8.02 s, total: 1min 24s
Wall time: 39.2 s
